In [8]:
pip install kaggle torch numpy matplotlib pandas

  Using cached pandas-2.3.1-cp312-cp312-win_amd64.whl.metadata (19 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pandas-2.3.1-cp312-cp312-win_amd64.whl (11.0 MB)
Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
Using cached tzdata-2025.2-py2.py3-none-any.whl (347 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
import torch
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

device = torch.cuda.is_available()
device

False

Loading Data

In [14]:
train_file_path = r"C:\Users\amanb\Ml Projects - 20\titanic\titanic\train.csv"
test_file_path = r"C:\Users\amanb\Ml Projects - 20\titanic\titanic\train.csv"
train = pd.read_csv(train_file_path)
test = pd.read_csv(test_file_path)

Converting Non numerical values to numerical values for machine learning

In [33]:
y_train = train['Survived']
y_train = y_train.to_numpy(dtype="int64")
features = ['Pclass','Sex','Age','Cabin','Parch']

x_test = pd.get_dummies(test[features],drop_first=True,dtype="int64")
x_test = x_test.to_numpy()
x_train = pd.get_dummies(train[features],drop_first=True,dtype="int64")
x_train = x_train.to_numpy()
y_test = test['Survived']
y_test = y_test.to_numpy(dtype="int64")
y_train_tensor = torch.from_numpy(y_train).type(torch.float64).type(torch.float64)
y_test_tensor = torch.from_numpy(y_test).type(torch.float64).type(torch.float64)

x_train_tensor = torch.from_numpy(x_train).type(torch.float64)
x_test_tensor = torch.from_numpy(x_test).type(torch.float64)
x_test_tensor,y_test_tensor

(tensor([[ 3., 22.,  0.,  ...,  0.,  0.,  0.],
         [ 1., 38.,  0.,  ...,  0.,  0.,  0.],
         [ 3., 26.,  0.,  ...,  0.,  0.,  0.],
         ...,
         [ 3., nan,  2.,  ...,  0.,  0.,  0.],
         [ 1., 26.,  0.,  ...,  0.,  0.,  0.],
         [ 3., 32.,  0.,  ...,  0.,  0.,  0.]], dtype=torch.float64),
 tensor([0., 1., 1., 1., 0., 0., 0., 0., 1., 1., 1., 1., 0., 0., 0., 1., 0., 1.,
         0., 1., 0., 1., 1., 1., 0., 1., 0., 0., 1., 0., 0., 1., 1., 0., 0., 0.,
         1., 0., 0., 1., 0., 0., 0., 1., 1., 0., 0., 1., 0., 0., 0., 0., 1., 1.,
         0., 1., 1., 0., 1., 0., 0., 1., 0., 0., 0., 1., 1., 0., 1., 0., 0., 0.,
         0., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 0., 1., 1., 0., 0., 1., 0.,
         0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 1., 1.,
         0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1.,
         0., 1., 1., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 1., 0.,
         0., 0., 1., 0., 0., 0., 

In [22]:
from torch import nn

class SurvivalClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer_1 = nn.Linear(5,8)
        self.layer_2 = nn.Linear(8,16)
        self.layer_3 = nn.Linear(16,8)
        self.layer_4 = nn.Linear(8,1)
        self.relu = nn.ReLU()
    def forward(self,x):
        return self.layer_4(self.relu(self.layer_3(self.relu(self.layer_2(self.relu(self.layer_1(x)))))))
model = SurvivalClassifier()
model

SurvivalClassifier(
  (layer_1): Linear(in_features=5, out_features=8, bias=True)
  (layer_2): Linear(in_features=8, out_features=16, bias=True)
  (layer_3): Linear(in_features=16, out_features=8, bias=True)
  (layer_4): Linear(in_features=8, out_features=1, bias=True)
  (relu): ReLU()
)

In [23]:
from torch import nn
loss_fn = nn.BCEWithLogitsLoss()
optim = torch.optim.SGD(params=model.parameters(),lr= 0.3)

In [24]:
def accuracy_fn(y_true,y_preds):
  correct = torch.eq(y_true,y_preds).sum().item()
  accur = (correct/len(y_preds))*100
  return accur

In [34]:
epochs = 10

for epoch in range(epochs):
    model.train()
    y_logits = model(x_train_tensor).squeeze()
    y_preds = torch.round(torch.sigmoid(y_logits))
    loss = loss_fn(y_logits,y_train_tensor)
    optim.zero_grad()
    loss.backwards()
    optim.step()
    model.eval()
    with torch.inference_mode():
        y_logits = model(x_test_tensor).squeeze()
        y_preds = torch.round(torch.sigmoid(y_logits))
        acc = accuracy_fn(y_test_tensor,y_preds)
        loss = loss_fn(y_logits,y_test_tensor)
    print(f"Acc:{acc} || Loss : {loss}")

RuntimeError: mat1 and mat2 must have the same dtype, but got Double and Float